In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import *
from constants import *
from imports import *
from loaders import *
from train_utils import *
from rollout import RwEvaluator

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

('1.13.1', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'2ljwym18'

In [3]:
wandb.init(id='372aji4n', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet().to(device) 
m.train()
sum([torch.numel(p) for p in m.parameters()]) / 1000

30484.11

In [5]:
m = try_load_state_dict(m, torch.load(f"{BESPOKE_ROOT}/models/m.torch"))

In [6]:
%%time
from rollout import RwEvaluator
rw_evaluator = RwEvaluator(m, wandb=wandb)

CPU times: user 18 µs, sys: 8 µs, total: 26 µs
Wall time: 28.6 µs


In [7]:
model_stem = "3.7"

In [8]:
%%time

rnn_only = False
rnn_is_first = False

if rnn_only: # m cnn backbone needs to be all loaded up and ready

    if rnn_is_first:
        freeze_model(m, True)
        freeze_model(m.rnn, False)
        m.hidden_init.requires_grad = True
        m.cell_init.requires_grad = True
    else:
        freeze_model(m.backbone, True)
        
    m.backbone.train(False) # freezing bn params
    bs = 16
    bptt = 42 #21 #7 # these have to be multiple of Zloader's internal bptt, currently 3 for bs 32 or 7 for bs 16
    seqlen = bptt*10 #5 #15 
    constant_seq_aug = .0 #TODO can only do constant aug when bptt is one, currently. Fix this.
    n_workers = 5 if seqlen > 100 else 4 if seqlen > 400 else 8 # maxxing out ram here w higher bs. Was 8 w bs 16
    # fewer than five workers we start to have to wait too long for chunk. More is better here.
    dataloader_trn = ZLoader(m, bs=bs, bptt=bptt, seqlen=seqlen, constant_seq_aug=constant_seq_aug, n_workers=n_workers)
else:
    bs = 32
    bptt = 1
    seqlen = 1
    constant_seq_aug = 0 #.9 TODO temp disabled
    n_workers = 8
    dataloader_trn = TrnLoader(bs=bs, bptt=bptt, seqlen=seqlen, constant_seq_aug=constant_seq_aug, n_workers=n_workers)
    
lr = 3e-4
m.use_rnn = rnn_only

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been depreca

CPU times: user 62.4 ms, sys: 1.05 s, total: 1.11 s
Wall time: 1.37 s


/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


In [9]:
opt = torch.optim.Adam(m.parameters(), lr=lr)
#opt = torch.optim.AdamW(m.parameters(), lr=3e-4, weight_decay=.001)

In [10]:
m.backbone.training, m.training

(True, True)

In [11]:
[(n,p.requires_grad) for (n,p) in m.named_parameters()]

[('hidden_init', True),
 ('cell_init', True),
 ('backbone.conv_stem.weight', True),
 ('backbone.bn1.weight', True),
 ('backbone.bn1.bias', True),
 ('backbone.blocks.0.0.conv_dw.weight', True),
 ('backbone.blocks.0.0.bn1.weight', True),
 ('backbone.blocks.0.0.bn1.bias', True),
 ('backbone.blocks.0.0.se.conv_reduce.weight', True),
 ('backbone.blocks.0.0.se.conv_reduce.bias', True),
 ('backbone.blocks.0.0.se.conv_expand.weight', True),
 ('backbone.blocks.0.0.se.conv_expand.bias', True),
 ('backbone.blocks.0.0.conv_pw.weight', True),
 ('backbone.blocks.0.0.bn2.weight', True),
 ('backbone.blocks.0.0.bn2.bias', True),
 ('backbone.blocks.0.1.conv_dw.weight', True),
 ('backbone.blocks.0.1.bn1.weight', True),
 ('backbone.blocks.0.1.bn1.bias', True),
 ('backbone.blocks.0.1.se.conv_reduce.weight', True),
 ('backbone.blocks.0.1.se.conv_reduce.bias', True),
 ('backbone.blocks.0.1.se.conv_expand.weight', True),
 ('backbone.blocks.0.1.se.conv_expand.bias', True),
 ('backbone.blocks.0.1.conv_pw.weight

In [12]:
%%time
"""m.model_stem = model_stem
rw_evaluator.evaluate()"""

CPU times: user 4 µs, sys: 3 µs, total: 7 µs
Wall time: 16.2 µs


'm.model_stem = model_stem\nrw_evaluator.evaluate()'

In [13]:
for n, mm in m.backbone.named_modules():
    if type(mm) == nn.BatchNorm2d: 
        mm.momentum = .99 #.4

In [17]:
trainer = Trainer(dataloader_trn, m, model_stem=model_stem, opt=opt, wandb=wandb, 
                  rw_evaluator=rw_evaluator, rnn_only=rnn_only)

In [15]:
trainer.reload_state()

Loading model from /home/beans/bespoke/models/m.torch. 
Last modified 10845 min ago.
Loading opt from /home/beans/bespoke/models/opt.torch. 
Last modified 10844 min ago.
Currently at epoch 79.


In [18]:
copy_cnn_params = False
if copy_cnn_params:
    m.copy_cnn_params_to_rnn() # when first trn rnn, has to be after reload_state.
    print("copied")
    
if rnn_only:
    trainer.loss_manager.update_emas = False
    trainer.loss_manager.loss_emas['te'] = .002 # manual. Bc cnn doesn't cache this.

In [19]:
trainer.loss_manager.update_emas

True

In [20]:
trainer.loss_manager.loss_weights

{'has_stop': 0.1,
 'stop_dist': 0.1,
 'has_lead': 0.1,
 'lead_dist': 0.1,
 'lead_speed': 0.02,
 'lane_width': 0.02,
 'dagger_shift': 0.05,
 'rd_is_lined': 0.02,
 'left_turn': 0.01,
 'right_turn': 0.01,
 'td': 0.03,
 'pitch': 0.1,
 'yaw': 0.1,
 'unc': 0.1,
 'te': 0.05,
 'wp_angles': 1,
 'wp_angles_i': 0.5,
 'wp_curvatures': 0.2,
 'wp_curvatures_i': 0.1,
 'wp_headings': 0.02,
 'wp_headings_i': 0.01,
 'wp_rolls': 0.05,
 'wp_rolls_i': 0.025,
 'wp_zs': 0.02,
 'wp_zs_i': 0.01}

In [21]:
trainer.loss_manager.loss_emas

{'has_stop': 10.0,
 'stop_dist': 10.0,
 'has_lead': 10.0,
 'lead_dist': 10.0,
 'lead_speed': 10.0,
 'lane_width': 10.0,
 'dagger_shift': 10.0,
 'rd_is_lined': 10.0,
 'left_turn': 10.0,
 'right_turn': 10.0,
 'td': 10.0,
 'pitch': 10.0,
 'yaw': 10.0,
 'unc': 10.0,
 'te': 10.0,
 'wp_angles': 1.0,
 'wp_angles_i': 10.0,
 'wp_curvatures': 10.0,
 'wp_curvatures_i': 10.0,
 'wp_headings': 10.0,
 'wp_headings_i': 10.0,
 'wp_rolls': 10.0,
 'wp_rolls_i': 10.0,
 'wp_zs': 10.0,
 'wp_zs_i': 10.0}

In [ ]:
trainer.train()

180.625 2698.0 314.5 29.5 497.75 414.0 37.125 150.5 3.8671875 121.375
wp_angles_i tensor(3.1465, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings_i tensor(3.2480, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures_i tensor(8.4297, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls_i tensor(0.0131, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs_i tensor(0.1046, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_angles tensor(3.4102, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings tensor(0.4150, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures tensor(2.6934, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls tensor(3.0625, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs tensor(3.4297, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
has_stop tensor(0.0192, device='cuda:0

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/plotly/matplotlylib/mplexporter/exporter.py:289: MatplotlibDeprecationWarning:


The get_offset_position function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.



211.125 212.25 17.96875 339.75 10.2578125 533.0 164.625 111.0625 2.3671875 854.5
92.625 96.875 10.0078125 6.4140625 6.4453125 4.53125 3.369140625 268.25 0.76416015625 391.5
111.3125 1586.0 145.75 14.90625 334.25 3.955078125 21.5 314.5 2.623046875 365.0
61.53125 130.125 7.4765625 12.765625 20.625 2.423828125 34.1875 251.375 0.8447265625 1033.0
wp_angles_i tensor(0.1527, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings_i tensor(0.0981, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures_i tensor(1.2422, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls_i tensor(0.0018, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs_i tensor(0.1458, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_angles tensor(0.1057, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings tensor(0.1837, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures tensor(0.820

23.40625 116.1875 13.1953125 9.671875 8.359375 5.66015625 22.6875 114.875 0.2269287109375 124.4375
wp_angles_i tensor(0.1445, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings_i tensor(0.1008, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures_i tensor(0.6836, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls_i tensor(0.0028, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs_i tensor(0.0911, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_angles tensor(0.1186, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings tensor(0.1296, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures tensor(0.4326, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls tensor(6.7188, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs tensor(4.4414, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
has_stop 

245.75 213.5 19.828125 21.28125 66.0 8.5 8.84375 95.375 3.29296875 495.0


 {'logistical/obs_consumed_per_second': 39.2578125, 'logistical/obs_generated_per_second': 61.953125, 'logistical/slowest_runner_obs_per_sec': 6.8671875, 'logistical/data_consumption_ratio': 0.63490401, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 0.00105846, 'timing/model forward': 0.18211814, 'timing/calc losses': 0.00485558, 'timing/backwards': 0.62133918, 'timing/get worst': 0.00286828, 'timing/logging': 0.01932009, 'timing/calc timing': 0.00242187, 'timing/trn update': 0.83398476, 'avg_unc': -2.50955963, 'wp_angles_i': 0.009312, 'wp_headings_i': 0.00743825, 'wp_curvatures_i': 0.03760886, 'wp_rolls_i': 0.0001497, 'wp_zs_i': 0.00228718, 'wp_angles': 0.00210603, 'wp_headings': 0.00257952, 'wp_curvatures': 0.01498421, 'wp_rolls': 0.04033735, 'wp_zs': 0.05513504, 'has_stop': 0.014375, 'stop_dist': 0.00179131, 'has_lead': 0.01576527, 'lead_dist': 0.00291145, 'lead_speed': 0.04298401, 

...



 {'logistical/obs_consumed_per_second': 37.3203125, 'logistical/obs_generated_per_second': 55.4140625, 'logistical/slowest_runner_obs_per_sec': 5.09375, 'logistical/data_consumption_ratio': 0.6748506, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 0.00174278, 'timing/model forward': 0.20867676, 'timing/calc losses': 0.00515007, 'timing/backwards': 0.63528042, 'timing/get worst': 0.00342144, 'timing/logging': 0.01648374, 'timing/calc timing': 0.00271151, 'timing/trn update': 0.8734703, 'avg_unc': -2.9563446, 'wp_angles_i': 0.00545461, 'wp_headings_i': 0.00392564, 'wp_curvatures_i': 0.02197331, 'wp_rolls_i': 0.0001352, 'wp_zs_i': 0.00143113, 'wp_angles': 0.00099811, 'wp_headings': 0.00113924, 'wp_curvatures': 0.00725705, 'wp_rolls': 0.02979439, 'wp_zs': 0.04339386, 'has_stop': 0.00798756, 'stop_dist': 0.00127405, 'has_lead': 0.00944617, 'lead_dist': 0.00140166, 'lead_speed': 0.03709869, 'dagger_shift': 0.03587288, 'lane_width': 0.04971617, 'rd_is_lined': 0.00

22.90625 34.1875 0.666015625 2.7734375 0.529296875 5.05078125 16.828125 246.125 0.134521484375 78.8125
14.0390625 91.625 6.0546875 4.36328125 26.203125 1.625 24.78125 86.5 0.71533203125 103.625
35.875 135.5 23.96875 6.125 17.796875 1.4189453125 27.609375 431.25 0.3359375 212.25


 {'logistical/obs_consumed_per_second': 37.2734375, 'logistical/obs_generated_per_second': 56.03125, 'logistical/slowest_runner_obs_per_sec': 7.0, 'logistical/data_consumption_ratio': 0.66733641, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 0.00178942, 'timing/model forward': 0.20926348, 'timing/calc losses': 0.00557698, 'timing/backwards': 0.63553527, 'timing/get worst': 0.00247528, 'timing/logging': 0.01657444, 'timing/calc timing': 0.00299527, 'timing/trn update': 0.8742139, 'avg_unc': -2.93437195, 'wp_angles_i': 0.00441796, 'wp_headings_i': 0.00325507, 'wp_curvatures_i': 0.01825006, 'wp_rolls_i': 5.9e-05, 'wp_zs_i': 0.00146289, 'wp_angles': 0.00108108, 'wp_headings': 0.00131191

259.5 39.6875 29.90625 4.95703125 34.3125 15.5625 1.2451171875 194.5 1.390625 383.5
25.640625 280.0 14.5 2.9765625 8.9140625 1.763671875 3.34375 235.5 0.047607421875 98.5
74.4375 147.5 12.0859375 49.78125 25.78125 149.0 1.3447265625 217.625 0.058135986328125 131.5
36.4375 62.25 29.90625 2.564453125 73.8125 6.6484375 3.5546875 145.625 44.3125 147.25
29.296875 399.5 40.9375 1.513671875 67.9375 10.3203125 48.0 427.25 0.348388671875 76.6875
wp_angles_i tensor(0.2328, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings_i tensor(0.1447, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures_i tensor(1.4873, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls_i tensor(0.0033, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs_i tensor(0.2396, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_angles tensor(0.0659, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings tensor(0

6.7734375 21.96875 4.44921875 2.919921875 20.171875 16.59375 0.5068359375 164.25 0.11614990234375 90.75
33.34375 46.1875 7.4765625 4.41015625 9.046875 3.65234375 22.75 176.125 0.496337890625 51.5
44.0 172.25 104.8125 4.58203125 100.0 12.359375 3.533203125 82.125 0.033905029296875 224.375
wp_angles_i tensor(1.5801, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings_i tensor(1.1543, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures_i tensor(3.0293, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls_i tensor(0.0006, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs_i tensor(0.0751, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_angles tensor(0.1080, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings tensor(0.0873, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures tensor(0.5200, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBa



 {'logistical/obs_consumed_per_second': 38.46875, 'logistical/obs_generated_per_second': 58.0546875, 'logistical/slowest_runner_obs_per_sec': 6.0, 'logistical/data_consumption_ratio': 0.66305799, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 0.00092281, 'timing/model forward': 0.19419498, 'timing/calc losses': 0.00484902, 'timing/backwards': 0.6261948, 'timing/get worst': 0.00206426, 'timing/logging': 0.03697979, 'timing/calc timing': 0.00247287, 'timing/trn update': 0.86768175, 'avg_unc': -2.9165802, 'wp_angles_i': 0.00542541, 'wp_headings_i': 0.00328112, 'wp_curvatures_i': 0.0180552, 'wp_rolls_i': 7.037e-05, 'wp_zs_i': 0.00124143, 'wp_angles': 0.00110798, 'wp_headings': 0.0012918, 'wp_curvatures': 0.00730947, 'wp_rolls': 0.03421471, 'wp_zs': 0.04566032, 'has_stop': 0.01133043, 'stop_dist': 0.00096511, 'has_lead': 0.00694271, 'lead_dist': 0.00098658, 'lead_speed': 0.03134985, 'dagger_shift': 0.04358084, 'lane_width': 0.04719901, 'rd_is_lined': 0.00182779,

In [22]:
t = torch.randn(3,4,5)
t[0,0,0] = torch.inf

In [23]:
t

tensor([[[    inf, -0.7854, -1.3716,  0.8498, -0.0986],
         [ 0.3141, -1.5963, -0.3781, -1.1632,  0.0405],
         [-0.0689, -0.9696,  1.4334,  1.0205,  2.2483],
         [ 1.2481, -0.8255, -0.5951, -0.0256,  0.2744]],

        [[ 0.6463,  1.3897,  0.8417, -0.3971, -1.4846],
         [ 0.3036, -0.4071, -0.0882, -0.6457,  0.2220],
         [ 1.5495,  0.6217, -0.4204,  0.3186,  0.4908],
         [-2.7397, -2.3005,  1.9402, -0.5320, -1.1312]],

        [[ 0.2612,  0.7503, -0.6469,  0.5444, -0.6493],
         [-1.0282, -1.1678, -0.1762,  1.1714,  0.2500],
         [ 0.6518, -0.5325, -1.2205,  0.8746,  0.5583],
         [ 0.3824, -0.7756,  0.8413,  0.3293, -1.3277]]])

In [24]:
t.max()

tensor(inf)

In [20]:
AUX_TARGET_PROPS

['has_stop',
 'stop_dist',
 'has_lead',
 'lead_dist',
 'lead_speed',
 'dagger_shift',
 'lane_width',
 'rd_is_lined',
 'left_turn',
 'right_turn']

In [20]:
trainer.loss_manager.loss_emas

{'has_stop': 0.6581827152218972,
 'stop_dist': 0.1914074304465102,
 'has_lead': 0.6045025000545183,
 'lead_dist': 0.18079276826857715,
 'lead_speed': 0.8834656769419045,
 'lane_width': 4.096574170043749,
 'dagger_shift': 2.512469867257369,
 'rd_is_lined': 0.10581200184235365,
 'left_turn': 0.08856740587054233,
 'right_turn': 0.07401701428385941,
 'td': 10.0,
 'pitch': 10.0,
 'yaw': 10.0,
 'unc': 117.34792707051348,
 'te': 0.002,
 'wp_angles': 0.0859911379087128,
 'wp_angles_i': 4.15318218310992,
 'wp_curvatures': 0.7827851002866839,
 'wp_curvatures_i': 17.866540970943753,
 'wp_headings': 0.12228421566478304,
 'wp_headings_i': 2.901144808827211,
 'wp_rolls': 4.211842307760256,
 'wp_rolls_i': 0.13459417300430104,
 'wp_zs': 5.043518468202303,
 'wp_zs_i': 1.1309414719989206}

In [20]:
m.hidden_init, m.cell_init

(Parameter containing:
 tensor([[[ 0.0015, -0.0488, -0.0292,  ..., -0.0560, -0.0283,  0.0664]]],
        device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([[[-0.3869,  0.0330,  0.0708,  ..., -0.0204,  0.3181, -0.2530]]],
        device='cuda:0', requires_grad=True))

In [21]:
m.training, m.backbone.training 

(True, False)

In [22]:
set_trainer_should_stop(False)

In [28]:
{k:trainer.loss_manager.loss_emas["wp_angles"]/trainer.loss_manager.loss_emas[k] for k,v in trainer.loss_manager.loss_emas.items()}

{'has_stop': 29.734797363340526,
 'stop_dist': 87.87280383313023,
 'has_lead': 40.201898525296386,
 'lead_dist': 127.76564504863305,
 'lead_speed': 0.34846566491395553,
 'lane_width': 3.809900606793288,
 'dagger_shift': 4.134576796863424,
 'rd_is_lined': 16457.381372040072,
 'left_turn': 125.55969474655697,
 'right_turn': 283.74133223646584,
 'td': 0.015300805875045518,
 'pitch': 0.015300805875045518,
 'yaw': 0.015300805875045518,
 'unc': 0.11512757866420895,
 'te': 61.54656219577027,
 'wp_angles': 1.0,
 'wp_angles_i': 0.240031243936724,
 'wp_curvatures': 0.16382346075192508,
 'wp_curvatures_i': 0.003522151816451157,
 'wp_headings': 0.5183519273228973,
 'wp_headings_i': 0.005565440760627469,
 'wp_rolls': 0.027721036050055964,
 'wp_rolls_i': 8.663331955712781,
 'wp_zs': 0.00020285505128895626,
 'wp_zs_i': 0.0035411505624853203}

In [23]:
{'has_stop': 0.005145757574225003,
 'stop_dist': 0.001741244754645775,
 'has_lead': 0.0038059908701619494,
 'lead_dist': 0.0011975680840668381,
 'lead_speed': 0.4390907746627964,
 'lane_width': 0.04016064316156499,
 'dagger_shift': 0.0370069456362572,
 'rd_is_lined': 9.297229935401817e-06,
 'left_turn': 0.0012186080816722507,
 'right_turn': 0.0005392519219686348,
 'td': 10.0,
 'pitch': 10.0,
 'yaw': 10.0,
 'unc': 1.329030459302299,
 'te': 0.002486053701322257,
 'wp_angles': 0.15300805875045517,
 'wp_angles_i': 0.6374505928519476,
 'wp_curvatures': 0.9339813604728601,
 'wp_curvatures_i': 1.44164213359285,
 'wp_headings': 0.2951818073498582,
 'wp_headings_i': 1.49253209788985,
 'wp_rolls': 5.519564942456264,
 'wp_rolls_i': 0.017661571729288113,
 'wp_zs': 754.2728553133405,
 'wp_zs_i': 43.208571917673005


import sys
sys.getsizeof(trainer.loss_manager.loss_emas)

1176

In [16]:
torch.save(trainer.model.state_dict(), f"{BESPOKE_ROOT}/models/m{trainer.model_stem}_e{trainer.current_epoch}.torch")

In [16]:
m.training

Error in callback <function _WandbInit._resume_backend at 0x7f62ef023160> (for pre_run_cell):


Exception: The wandb backend process has shutdown

True

Error in callback <function _WandbInit._pause_backend at 0x7f62e3b863a0> (for post_run_cell):


Exception: The wandb backend process has shutdown

In [ ]:
trainer.opt.param_groups[0]['lr'] = 3e-4

In [28]:
snr

0.15067877149872744



 {'logistical/obs_consumed_per_second': 38.2265625, 'logistical/obs_generated_per_second': 85.1328125, 'logistical/slowest_runner_obs_per_sec': 5.3125, 'logistical/data_consumption_ratio': 0.45004667, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 0.00400971, 'timing/model forward': 0.19439457, 'timing/calc losses': 0.00539116, 'timing/backwards': 1.11790375, 'timing/get worst': 0.00264816, 'timing/logging': 0.01570235, 'timing/calc timing': 0.01359244, 'timing/trn update': 1.35364565, 'wp_angles': 0.01003864, 'wp_curvatures': 0.04547036, 'wp_headings': 0.01048879, 'wp_rolls': 0.52840137, 'wp_zs': 0.09037566, 'has_stop': 0.01459547, 'stop_dist': 0.00825622, 'has_lead': 0.01966284, 'lead_dist': 0.00662124, 'lead_speed': 16.11909485, 'dagger_shift': 0.14152849, 'lane_width': 0.12975717, 'rd_is_lined': 0.01347083, 'pitch': 4.037e-05, 'yaw': 4.98e-05, 'unc': 1.57641983, 'logistical/max_param': 32.69097137, 'logistical/lr': 0.0006, 'logistical/mins_since_slowest

In [9]:
%%time

rnn_only = True

if rnn_only: # m cnn backbone needs to be all loaded up and ready
    #freeze_model(m, True)
    #unfreeze_part_of_model(m, "rnn")
    freeze_model(m.backbone, True)
    m.backbone.train(False) # freezing bn params
    bs = 16
    bptt = 14 #35 #8 #7
    seqlen = bptt*5 # needs to be mult of 7
    constant_seq_aug = .0 #TODO can only do constant aug when bptt is one, currently. Fix this.
    dataloader_trn = ZLoader(m, bs=bs, bptt=bptt, seqlen=seqlen)
else:
    bs = 32
    bptt = 1
    seqlen = 1
    constant_seq_aug = .9
    dataloader_trn = TrnLoader(bs=bs, bptt=bptt, seqlen=seqlen, constant_seq_aug=constant_seq_aug)

Launching 3 loader workers


/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated.

Got first chunk
CPU times: user 1.1 s, sys: 5.72 s, total: 6.83 s
Wall time: 45.2 s


Process Process-3:
Process Process-4:
Process Process-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/beans/bespoke/loaders.py", line 133, in make_chunks
    time.sleep(.1)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 1

In [25]:
set_should_stop(True)

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
